In [78]:
import sys
sys.path.append("..")
from MOF_build.functions.read import read
from MOF_build.functions.output import output
from MOF_build.functions.normv import normalize_vector
from MOF_build.functions.rotate import rotate
import pandas as pd

def get_new_methyl_cut(cut):
    C1 = cut.loc[2, ["x", "y", "z"]].to_numpy()
    CC = normalize_vector(C1) 
    methyl=read.pdb('methyl.pdb')
    methylH1=methyl.loc[1, ["x", "y", "z"]].to_numpy()
    methylH2=methyl.loc[2, ["x", "y", "z"]].to_numpy()
    methylH3=methyl.loc[3, ["x", "y", "z"]].to_numpy()
    methylCH = (methylH1+methylH2+methylH3)/3 #methyl C is 0 0 0 
    p1 = methylCH
    p2 = C1
    methyl_arr = methyl.loc[:, ["x", "y", "z"]].to_numpy()

    q=rotate.calculate_q_rotation_with_vectors(p1,p2)

    new_methyl=rotate.get_rotated_array(methyl_arr,q)+1.5*CC+C1 #set c-c as 1.5 A
    df_carbo = cut.iloc[0:3,:]

    methyl.loc[:, ["x", "y", "z"]] = new_methyl
    df_newcut = pd.concat([df_carbo,methyl],ignore_index=True,join='outer')
    df_newcut.loc[:,'Residue']='CUT'
    
    return df_newcut

In [79]:
methyl=read.pdb('methyl.pdb')
terms=read.pdb('forcutPCN222.pdb')

#for 0-8 CO2+methyl terminations
for i in range(8):
    start = 3*i
    cut=terms[start:start+3].reset_index(drop=True)
    if i ==0:
        init=get_new_methyl_cut(cut)
        init.loc[:,'Res_number']= i+1
        df_cut = init
    else:
        new_cut_df=get_new_methyl_cut(cut)
        new_cut_df.loc[:,'Res_number']= i+1
        df_cut=pd.concat([df_cut,new_cut_df],join='outer',ignore_index=True)


In [80]:
'''translate and rotate cut '''
import numpy as np

array = df_cut.loc[:, ['x','y','z']].to_numpy()
center = 0.5*(array[14]+array[28]) #center point (beginning point)
new_node = array-center


#PCN222
any_tdx, any_tdy, any_tdz = (
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([0, 1, 1]))
dx, dy, dz = np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])
carte_basis = np.vstack([dx, dy, dz])
  
tric_basis = rotate.coordinate_transfer(any_tdx, any_tdy, any_tdz, carte_basis)

v1_file = new_node[37]-new_node[44]
v2_file = new_node[44]-new_node[30]
v1_frame = tric_basis[2]+tric_basis[1]
v2_frame = tric_basis[0]
rotated_node0 = rotate.rotate_twice_linker(df_cut,center,v1_file,v1_frame,v2_file,v2_frame)


rotateq_z45=rotate.calculate_q_rotation_with_axis_degree(dz,np.pi/4)
rotated_node1 = rotate.get_rotated_array(rotated_node0,rotateq_z45)
#rotateq_x90=rotate.calculate_q_rotation_with_axis_degree(dx,np.pi)
q=rotate.calculate_q_rotation_with_vectors((rotated_node1[15]-rotated_node1[50]),dx)
rotated_node = rotate.get_rotated_array(rotated_node1,q)
df_cut.loc[:, ['x','y','z']] = rotated_node


In [81]:
df_left = df_cut[["Atom_label", "Residue", "Res_number", "Note"]]
df_right = df_cut[["x","y","z"]]
df = pd.concat([df_left, df_right], axis=1, join="outer")
df.to_csv("newlibcut.txt", header=None, sep="\t", index=False)
output('newlibcut',True,False,False)

In [82]:
from MOF_build.functions.clean import clean
clean.txt()
clean.xyz()
clean.gro()